In [1]:
import pandas as pd 
import numpy as np

In [2]:
#raws
raw_teleoperadoras=pd.read_excel(r"C:\Users\Abraham\Desktop\Solo\Bootcamp\portfolio\PowerBI\ventas_telemarketing.xlsx", skiprows=1)
raw_comerciales=pd.read_excel(r"C:\Users\Abraham\Desktop\Solo\Bootcamp\portfolio\PowerBI\ventas_comerciales.xlsx",skiprows=1)
raw_tecnicos=pd.read_excel(r"C:\Users\Abraham\Desktop\Solo\Bootcamp\portfolio\PowerBI\ventas_tecnicos.xlsx",skiprows=1)

## Tratado teleoperadoras

In [3]:
teleop=raw_teleoperadoras.copy()
teleop["Departamento"]="Teleoperadora"

In [4]:
#extraigo fuente de donde provienen las ventas:
import unicodedata

def normaliza(texto):
    texto = str(texto).lower()
    texto = unicodedata.normalize('NFKD', texto)
    texto = ''.join(c for c in texto if not unicodedata.combining(c))
    return texto

def fuente(text):
    text = normaliza(text)
    if "web" in text:
        return "Web"
    elif "hoy" in text:
        return "Reformashoy"
    elif "chat" in text:
        return "Web"
    elif "renova" in text:
        return "renova"
    elif "mail" in text:
        return "Web"
    elif "pisos" in text:
        return "Rdp"
    else:
        return "otros"

def fuente (text):
    text=str(text).lower()
    if "web" in text:
        return "Web"
    elif "hoy" in text:
        return "Reformashoy"
    elif "chat" in text:
        return "Web"
    elif "renova" in text:
        return "Renova"
    elif "mail" in text:
        return "Web"
    elif "pisos" in text:
        return "Rdp"
    else:
        return "Otros"

In [5]:
teleop["Fuente"]=teleop["Comercial/Telemarketing/Técnico"].apply(fuente)

In [6]:
#Agrupo entre externos, internos por web y sin web

def origen(text):
    text=str(text).lower()
    if "renova" in text:
        return "externos"
    elif "reformashoy" in text:
        return "externos"
    elif any(word in text for word in ["web", "chat", "mail", "reformasdepisos"]):
        return "internosweb"
    else: 
        return "internosnoweb"    
    
         

In [7]:
teleop["Origen"]=teleop["Fuente"].apply(origen)

In [8]:
def nombres (texto):
    texto=str(texto).lower()
    if "rosa" in texto:
     return "Rosa"
    elif "amparo" in texto:
     return "Amparo"
    elif "carmen" in texto:
      return "Maricarmen"
    elif "teresa" in texto:
      return "Teresa"
    elif "narcisa" in texto:
      return "Narcisa"
    elif "dolores" in texto:
      return "Loli"
    elif "loli" in texto:
      return "Loli"
    elif "monica" in texto:
      return "Monica"
    elif "vicky" in texto:
      return "Vicky"
    elif "natalia" in texto:
      return "Natalia"
    else: 
      return texto.capitalize()

In [9]:
teleop["Empleado"]=teleop["Comercial/Telemarketing/Técnico"].apply(nombres)

In [10]:
#Estos datos de teleoperadoras son clave para determinar el origen y la fuente en comerciales y técnicos
puente=teleop[["PTO","Fuente", "Origen"]]

In [11]:
puente.drop_duplicates(subset="PTO",inplace=True)


c:\Users\Abraham\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


## Comerciales

In [12]:
comercial=raw_comerciales.copy()

In [13]:
comercial=comercial.merge(puente,on="PTO",how="left")

In [14]:
comercial["Departamento"]="Comercial"
comercial["Empleado"]=comercial["Comercial/Telemarketing/Técnico"]

## Tecnicos

In [15]:
tecnico= raw_tecnicos.copy()

In [16]:
tecnico["Departamento"]="Tecnico"
tecnico["Empleado"]=tecnico["Comercial/Telemarketing/Técnico"]

In [17]:
tecnico=tecnico.merge(puente,on="PTO",how="left")

## Fusión de ficheros y clean

In [18]:
#comprobamos tienen el mismo número de columnas :Ok, 15
tecnico.shape

(2390, 15)

In [19]:
comercial.shape

(2449, 15)

In [20]:
teleop.shape

(2449, 15)

In [21]:
ventas_totales=pd.concat([teleop,comercial,tecnico],ignore_index=True)

In [22]:
#ventas_totales.drop(['Comercial/Telemarketing/Técnico','Observaciones Pagos' ],axis=1,inplace=True)

In [23]:
ventas_totales["Tipo"].unique()

array(['CALEFACCION VAILLANT', 'REFORMA VIVIENDA 5%',
       'REFORMA DE PINTURA', 'REFORMA DE COCINA',
       'REFORMA BAÑO + MAMPARA', 'CARPINTERIA DE MADERA',
       'REFORMA DE VIVIENDA 3%', 'REFORMA PINTURA Y VARIOS',
       'REFORMA DE VIVIENDA', 'REFORMA DE SUELO Y PINTURA',
       'TRABAJOS VARIOS', 'DESCUENTO DE OBRA', 'LICENCIA',
       'AMPLIACION DE OBRA', 'CONTENEDOR', 'REFORMA BAÑO',
       'AMPLIACION MUEBLES DE COCINA', 'MUEBLES DE COCINA',
       'LICENCIA Y CONTENEDOR', 'REFORMA COMUNIDAD',
       'REFORMA DE ELECTRICIDAD', 'REFORMA PLATO X PLATO',
       'REFORMA BAÑERA X PLATO', nan, 'AMPLIACION ALUMINIO / PVC',
       'BAJANTE COMUNIDAD', 'CALDERA', 'AIRE ACONDICIONADO VAILLANT',
       'REFORMA LOCAL', 'TRABAJOS DE FONTANERIA', 'ARREGLO MAMPARA',
       'MAMPARA', 'REFORMA TERRAZA', 'REFORMA PVC SLAVA',
       'MUEBLES DE COCINA 3% DCTO', 'REFORMA ALUMINIO NOGALES',
       'REFORMA DE COCINA 3%', 'REFORMA ALUMINIO SLAVA',
       'CALENTADOR VAILLANT', 'CALDERA VAI

In [24]:
def tipo (texto):
    texto=str(texto).lower()
    if "reforma" in texto:
        return "Reforma"
    elif "ampliacion" in texto:
        return "Ampliaciones"
    elif "descuento" in texto:
        return "Descuentos"
    elif "calefaccion" in texto:
        return "Calderas"
    elif "aire" in texto:
        return "Aires Acondicionados"
    elif "calentador" in texto:
        return "Calentadores"
    elif "licencia" in texto:
        return "Licencias"
    elif "contenedor" in texto:
        return "Contenedores"
    elif "interiorismo" in texto:
        return "Interiorismo"
    elif "trabajos" in texto:
        return "Reforma"
    else:
        return "Otros trabajos"

In [25]:
ventas_totales["Servicio"]=ventas_totales["Tipo"].apply(tipo)

In [33]:
ventas_totales.columns

Index(['Comercial/Telemarketing/Técnico', 'Fecha Acepta', 'F.P.R.',
       'Fecha Inicio', 'Fecha Entrega', 'PTO', 'Presupuesto Insitu',
       'Dirección', 'Importe', 'Observaciones Pagos', 'Tipo', 'Departamento',
       'Fuente', 'Origen', 'Empleado', 'Servicio'],
      dtype='object')

In [52]:
ventas_totales25=ventas_totales[ventas_totales["Fecha Acepta"].str.contains("2025")]

In [53]:
ventas_totales25["Column1"]=1

c:\Users\Abraham\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [58]:
ventas_totales25["FAcepta fto fecha"]=pd.to_datetime(ventas_totales['Fecha Acepta'],errors="coerce")

c:\Users\Abraham\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [59]:
ventas_totales25.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3689 entries, 5 to 7287
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   Comercial/Telemarketing/Técnico  3689 non-null   object        
 1   Fecha Acepta                     3689 non-null   object        
 2   F.P.R.                           3686 non-null   object        
 3   Fecha Inicio                     3278 non-null   object        
 4   Fecha Entrega                    3683 non-null   object        
 5   PTO                              3689 non-null   object        
 6   Presupuesto Insitu               3689 non-null   object        
 7   Dirección                        3689 non-null   object        
 8   Importe                          3689 non-null   float64       
 9   Observaciones Pagos              1409 non-null   object        
 10  Tipo                             3623 non-null   object     

In [70]:
filtro = ventas_totales25[ventas_totales25["FAcepta fto fecha"] < "2025-09-01"]

In [71]:
filtro.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3059 entries, 5 to 7287
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   Comercial/Telemarketing/Técnico  3059 non-null   object        
 1   Fecha Acepta                     3059 non-null   object        
 2   F.P.R.                           3056 non-null   object        
 3   Fecha Inicio                     2708 non-null   object        
 4   Fecha Entrega                    3053 non-null   object        
 5   PTO                              3059 non-null   object        
 6   Presupuesto Insitu               3059 non-null   object        
 7   Dirección                        3059 non-null   object        
 8   Importe                          3059 non-null   float64       
 9   Observaciones Pagos              1148 non-null   object        
 10  Tipo                             2993 non-null   object     

In [75]:
filtro.drop("FAcepta fto fecha",axis=1,inplace=True)

KeyError: "['FAcepta fto fecha'] not found in axis"

In [76]:
filtro

,Comercial/Telemarketing/Técnico,Fecha Acepta,F.P.R.,Fecha Inicio,Fecha Entrega,PTO,Presupuesto Insitu,Dirección,Importe,Observaciones Pagos,Tipo,Departamento,Fuente,Origen,Empleado,Servicio,Column1
5,AMPARO MONTOYA,03/03/2025,03/04/2025,28/03/2025,03/07/2024,65888,Sí,"C/ Puentelarra 8, Esc Izda 1º D - 28031 MADRID...",2306.00,"TOTAL 2.306 € + 21% IVA F.PAGO 10%,50%,40%",REFORMA BAÑO + MAMPARA,Teleoperadora,Otros,internosnoweb,Amparo,Reforma,1
7,AMPARO MONTOYA,23/01/2025,15/02/2025,31/03/2025,16/01/2025,66948,No,"C/ Camino de los Alamillos 14, 1º A - 28891 V...",21547.58,"TOTAL 21.547,58 + 10%IVA F.PAGO 10%, 30%, 40% ...",REFORMA DE VIVIENDA 3%,Teleoperadora,Otros,internosnoweb,Amparo,Reforma,1
8,AMPARO MONTOYA,18/02/2025,15/03/2025,10/04/2025,13/02/2025,66999,No,"C/ Europa Avda 17, Portal 8, 2º 2 - 28224 POZU...",16817.86,"TOTAL 16.817,86 €**CONTRATO PPAL DCTO 3% (50% ...",REFORMA DE VIVIENDA 3%,Teleoperadora,Otros,internosnoweb,Amparo,Reforma,1
13,AMPARO MONTOYA,30/07/2025,15/08/2025,NaN,30/07/2025,68263,Sí,"C/ Francisco Lozano 15, 3ºB - 28008 MADRID (MA...",5959.00,TOTAL= 5959 € + 21% IVA F.PAGO = FINANCIADO 36...,REFORMA DE SUELO Y PINTURA,Teleoperadora,Otros,internosnoweb,Amparo,Reforma,1
18,AMPARO MONTOYA,27/02/2025,15/03/2025,10/04/2025,13/02/2025,66999,No,"C/ Europa Avda 17, Portal 8, 2º 2 - 28224 POZU...",1233.10,NaN,AMPLIACION DE OBRA,Teleoperadora,Otros,internosnoweb,Amparo,Ampliaciones,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7283,Vasilia,27/08/2025,15/09/2025,21/08/2025,19/08/2025,68251,No,"C/ Pirra 18, Portal J 1º Izda - 28022 MADRID ...",798.00,NaN,AMPLIACION DE OBRA,Tecnico,Web,internosweb,Vasilia,Ampliaciones,1
7284,Vasilia,02/09/2025,17/09/2025,NaN,02/06/2025,67752,No,"C/ Saturno 58, Local - 28042 MADRID (MADRID)",88.00,NaN,AMPLIACION DE OBRA,Tecnico,Reformashoy,externos,Vasilia,Ampliaciones,1
7285,Vasilia,07/08/2025,13/10/2025,NaN,22/05/2025,67709,No,"C/ Gran Vía De Hortaleza 5E, 8ºB - 28033 MADR...",1271.67,NaN,AMPLIACION DE OBRA,Tecnico,Web,internosweb,Vasilia,Ampliaciones,1
7286,Vasilia,28/07/2025,13/10/2025,NaN,22/05/2025,67709,No,"C/ Gran Vía De Hortaleza 5E, 8ºB - 28033 MADR...",3729.65,DIFERENCIA ENTRE 1º PTO Y 2º PTO,AMPLIACION DE OBRA,Tecnico,Web,internosweb,Vasilia,Ampliaciones,1


In [77]:
filtro.to_excel("ventas2025.xlsx",index=False)